In [2]:
import tensorflow as tf
from gensim import models
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.model_selection import StratifiedKFold

In [21]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = vectorizer(np.array([[s] for s in train_x])).numpy()    
    val_x = vectorizer(np.array([[s] for s in val_x])).numpy()
    test_x = vectorizer(np.array([[s] for s in test_x])).numpy()

    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = layers.Embedding(input_dim = vocab_size, 
                                       output_dim = emb_dim, 
                                       embeddings_initializer=keras.initializers.Constant(matrix),
                                       trainable=True)
    x = embedding_layer(inputs)
    x = tf.keras.layers.Flatten()(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "logistic_models/{itr:04}".format(itr=cnt)
    checkpoint_path = model_path + "-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir logistic_models
    !mkdir $model_path
    
    model.compile(tf.optimizers.Adam(lr=0.001), "binary_crossentropy", metrics=[  "accuracy", 
                                                                                  tf.keras.metrics.AUC(name='auc')])
    
    

    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_auc', verbose=1,
            save_weights_only=True, save_best_only=True, mode='max')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )
    
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

Epoch 1/10
63/63 [==============================] - ETA: 0s - loss: 0.6809 - accuracy: 0.5679 - auc: 0.5787
Epoch 00001: val_auc improved from -inf to 0.66701, saving model to logistic_models/0001-0001.ckpt
63/63 [==============================] - 11s 179ms/step - loss: 0.6809 - accuracy: 0.5679 - auc: 0.5787 - val_loss: 0.6615 - val_accuracy: 0.6190 - val_auc: 0.6670
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 0.5248 - accuracy: 0.8025 - auc: 0.8934
Epoch 00002: val_auc improved from 0.66701 to 0.76389, saving model to logistic_models/0001-0002.ckpt
63/63 [==============================] - 11s 170ms/step - loss: 0.5248 - accuracy: 0.8025 - auc: 0.8934 - val_loss: 0.5932 - val_accuracy: 0.6910 - val_auc: 0.7639
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 0.2316 - accuracy: 0.9634 - auc: 0.9972
Epoch 00003: val_auc improved from 0.76389 to 0.81587, saving model to logistic_models/0001-0003.ckpt
63/63 [==============================] - 11s 

In [22]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))
print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))

print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))

test avg loss:  0.5452204346656799 +/- 0.03648450831747554
test avg acc:  0.7299999833106995 +/- 0.029812750588570573
test avg auc:  0.8053722620010376 +/- 0.027850594560996617


val avg loss:  0.5451412796974182 +/- 0.02221637279155503
val avg acc:  0.7353999853134155 +/- 0.015344057738416765
val avg auc:  0.8094303488731385 +/- 0.011165194584818966


train avg loss:  0.016808733716607092 +/- 0.013310216359561667
train avg acc:  0.9995500206947326 +/- 0.00028062732862639767
train avg auc:  0.9999986052513122 +/- 1.6674818840324545e-06
